# BMI 260 ASSIGNMENT #3 | Mammogram Spring 2018

## Name 1:Joseph Nicolls

## Name 2: Alex Lu

Breast cancer has the highest incidence and second highest mortality rate for women in the US. 
Your task is to utilize machine learning to either classify AND/OR segment mammograms or neither, as long as you justify why it is useful to do whatever it is you want to do. If someone turns in a deep dream assignment using mammograms this might be amusing, but not so useful to patients. Consider this a mini-project, we highly suggest you work with 1 other person, it can be someone in your team. 

In addition to the mammograms, the dataset includes segmentations and mass_case_description_train_set.csv, which contains information about the mass shape, mass margins, assessment number, pathology diagnosis, and subtlety. Take some time to research what all of these different fields mean and whether you can use them in your algorithm or not. You dont need to use all of what is provided to you. 

Some ideas:

1. use the ROI’s or segmentations to extract features, and then train a classifier based on those features using the algorithms presented to you in the machine learning lectures, does not need to be deep learning. 

2. use convolutional neural networks, feel free to use any of the code we went over in class or use your own (custom code, sklearn, keras, Tensorflow etc.). If you dont want to place helper functions and classes into this notebook, place them in a .py file in the same folder called helperfunctions.py and import them into this notebook. 

The data is here:

https://wiki.cancerimagingarchive.net/display/Public/CBIS-DDSM

If you do not like python, you can use a different language and turn your assignment in as a folder with all your code, a folder with all your figures and a latex or doc file with the writeup. The writeup doesnt need to be long, 1 page will do and cite at least one clinical paper and one technical paper. If you like python please place the writeup and code into this notebook. Use the markdown to tell is what you are doing in each section. You will not be graded on the performance of your model, only the scientific soundness of your claims, methodology, evaluation (ie fair but insightful statistics) and discussion of the shortcomings of what you tried. 

The format for this homework is as follows:

1. Describe what you are doing and why it matters to patients using at least one citation

2. Describe the relevant statistics of the data, how were the images taken ? how were they labeled ? what is the class balance and majority classifier accuracy ? How will you divide the data into testing, training and validation.

3. Describe your data pipeline, ie how is the data scrubbed, normalized, stored and fed to the model for training. 

4. Explain how the model you chose works along side the code for it and at least one technical citation to give credit where credit is due. 

5. There are many ways to do training, take us through how you do it. (ie we used early stopping and we decided when to stop based on validation loss)  

6. Make a figure displaying your results

7. Discuss pros and cons of your method and what you might have done differently now that youve tried or would try if you had more time. 

## Traditional ML techniques for Classification of Mammograms

###  Approach and Relevance

Mammograms are difficult to classify because identifying features for malignant tumors can be vague and masses in mammograms can appear anywhere with any orientation in the breast tissue. Though the setup for this problem suggests use of CNNs, we want to use traditional ML techniques in this exploratory research for a variety of reasons. The principal reason that we want to do this is for the feature analysis possible with traditional ML technqiues that isn't possible with CNNs. According to a study from Britton et. all, sensitivity of radiologists in classifying mammograms can vary between 53.1-74.1%. Through feature analysis, indiciative features could be highlighted for radiologists to focus on, potentially raising sensitivites and decreasing the variability of senestivities between radiologists. In essence, we will attempt to classify the malginancy of tumors within mammograms using traditional machine learning techniques based on quanitative features derived from the mammogram and categorical features provided by expert analysis on those mammograms. 




### Data Source


The images were taken from the Digital Database for Screening Mammography (DDSM), a database which consists of 2620 mammogram studies. These images are X-rays, recorded as grey-scale images. Previous work had been done to filter these images to the ROI mass. Within features_matrix.csv, results of calculations on these masses have been recorded. We have didvided the data into training and testing at random, leaving 10% out for testing. 52% of these images are negatively labeled while 48% are positively labeled. 

In addition, semantic features associated with some on the images have also been included. These semantic features include the type of view included in the image, which side of the patient the breast was on, and case descriptions with a limited vocabulary describing characteristics of the mass. We believe that semantic features such as breast density, mass shape, and mass margins can be highly useful. The limited vocabulary of these qualitative features allows one-hot encoding and incorporation into our feature vectors for traditional ML methods. 

Relevant statistics in our data are based on the conventional wisdom that malignant tumors typically have irregular borders and a non spherical shape. In order to measure this, we compare the margins of the mass to 
its convex hull, or the smallest set of convex points which contains the ROI. Two statistics which provide a metric of this relationshiop is the convexity (the ratio of the convex hull perimeter to the ROI perimeter) 
and the solidity ( the ratio of the ROI area to the convex hull area.) The lower the convexity and lower the solidity, the less spherical the ROI is, potentially hinting at malginant character. 

In addition, we have also calculated a similar metric called extent. This metric is the ratio of the area of the ROI to the area of the bounding box of the ROI. 

We will first try to leverage the following features in order to make attempts at classification: 
* features in the features_matrix.csv (ASM, Area, Centroid coordinates, ...) 
* convexity, solidity, and extent. (Calculated from masked ROI)


First, let's import the packages that we're going to use 

In [21]:
import h5py 
import numpy as np
import pandas as pd
import seaborn as sns
import cv2


In [22]:
import os 
import random
random.seed(42)

In [152]:
from sklearn import preprocessing

from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score, cross_val_predict

from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix

from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split

from skimage.morphology import convex_hull_image
from skimage import data, img_as_float
from skimage.util import invert
import matplotlib.pyplot as plt

from skimage.measure import regionprops


Defining global variables for paths, etc. 

In [24]:
validation_prop = .1


A function that calculates convex hull perimeter and area

In [107]:
def get_props(data): 
    ret,thresh_img = cv2.threshold(data,0,255,cv2.THRESH_BINARY)
    thresh_props =  regionprops(thresh_img.astype(int))
    thresh_area = thresh_props[0].area
    thresh_perimeter = thresh_props[0].perimeter
    chull = convex_hull_image(thresh_img)
    props = regionprops(chull.astype(int))
    chull_area = props[0].area
    chull_perimeter = props[0].perimeter 
    convexity = chull_perimeter/thresh_perimeter
    solidity = thresh_area/chull_area
    return convexity, solidity, thresh_props[0].extent, thresh_props[0].major_axis_length, thresh_props[0].minor_axis_length
    
    

A function that reads in image data

In [123]:
def gather_images(data_parent_dir):
    '''
        input:
            * paths, a list of the paths that we need to input 
        output:
            * a dataframe containing image data, label, and name 
    '''
    print("preparing to read images")
    
    # initialize data structures 
    data = []
    label = []
    name = []
    convexities = []
    solidities = []
    extents = []
    major_axis_lengths = []
    minor_axis_lengths = []
    
    paths = os.listdir(data_parent_dir)

    scan_ids = []
    
    # iteration over paths, dirs
    for first_path in paths:
        if first_path[0] == '.': # check for random dot files that come up :( 
            continue
        local_dir = os.path.join(data_parent_dir, first_path)
        for image in os.listdir(local_dir):
            scan_ids.append("_".join((first_path, image[:-3])))
            with h5py.File(os.path.join(local_dir, image), 'r') as hf:
                data.append(np.array(hf.get('data')))
                label.append(np.array(hf.get('label')).item(0))
                name.append(np.array(hf.get('name')))
                
                # compute additional features
                convexity, solidity, extent, major_axis_length, minor_axis_length = get_props(hf.get('data')[:, :, 1])
                convexities.append(convexity)
                solidities.append(solidity)
                extents.append(extent)
                major_axis_lengths.append(major_axis_length)
                minor_axis_lengths.append(minor_axis_length)
         
    print(scan_ids[:10])
    scan_ids = ["P_" +scan_id for scan_id in scan_ids]
    
    d = {'pixel_data':data, 'label':label, 'name':name}
    
    d_computed = {'convexity': convexities, 'solidity': solidities, 'extent': extents, 'major_axis_length': major_axis_lengths, 'minor_axis_length': minor_axis_lengths}
   
    df_img = pd.DataFrame(data=d, index=scan_ids)
    df_computed = pd.DataFrame(data=d_computed, index=scan_ids)

    print(len(df[df['label'] == 0]))
    print(len(df[df['label'] == 1]))
    print(len(df))
    
    return (df_img, df_computed, scan_ids)
            
            

In [124]:
def drop_excess_rows(scan_ids, precomputed_df):

    drop_list = []
    true_list = []
    scan_ids = set(scan_ids)
    names = precomputed_df.index.values
    for name in names:
        if name not in scan_ids:
            drop_list.append(name)

    print("the drop list has: " + str(len(drop_list)))
    return precomputed_df.drop(drop_list)

We initially predict some potential issues with this approach: the number of features is large, and many are potentially correlated. For that reason, we can implement some dimensionality reduction and take a look at the variation explained by the principal components (and their contents)

In [125]:
def reduce_dimensionality(raw_data, new_dims=3):
    '''
        input:
            * raw_data, the raw matrix that will be reduced in dimensionality 
        output:
            * the dimensionality-reduced data 
        
    '''
    print("preparing to reduce dimensionality")
    pca = PCA()
    pca.fit(raw_data)
    print(">>> variance explained by each principal component")
    print(pca.explained_variance_ratio_)  
    print(">>> the first principal component")
    print(pca.components_[0])
    reduced = pca.transform(raw_data)[:,:new_dims]
    return reduced

The beginnings of our preprocessing pipeline

In [141]:
def mean_center_normalize(data):
    data -= np.mean(data, axis = 0)
    data /= np.std(data, axis = 0)
    return data

def preprocess(data_parent_dir, precomputed_path, dim_reduc=None):
    print('Entering preprocessing')
    
    feature_names = []
    
    # reading things in 
    image_df, computed_df, scan_ids = gather_images(data_parent_dir)
    precomputed_df = drop_excess_rows(scan_ids, pd.read_csv(precomputed_path, index_col=0))

    # an initial merge
    feature_df = pd.merge(computed_df, precomputed_df, on=precomputed_df.index)
    
    # zero-centering, normalization of all data within the feature dataframe 
    features = feature_df.values[:,1:]
    print(features.shape)
    new_fts = mean_center_normalize(np.array(features,dtype=np.float32))
    print("==============================")
    print(new_fts.shape)
    
    print(image_df.describe())
    print(computed_df.describe())
    print(precomputed_df.describe())
    
    # dimensionality reduction 
    if dim_reduc is not None:
        new_fts = reduce_dimensionality(new_fts, dim_reduc)
        feature_names += ["pc# "+str(x) for x in range(1, dim_reduc+1)]
        feature_df = pd.DataFrame(new_fts, columns=feature_names, index=precomputed_df.index)
        print("==============================")
        print(feature_df.info())
        
    # adding back to df 
    else:
        feature_names += list(feature_df)[1:]
        feature_df[feature_names] = new_fts
        print("==============================")
        print(feature_df.info())
    
    print(feature_names)
    
    # joining dfs
    df = pd.merge(image_df, feature_df[feature_names], on=feature_df.index)
    df.info()
    
    train, test = train_test_split(df, test_size=0.2, random_state=260)
    
    return (train[feature_names], train['label'], test[feature_names], test['label'], scan_ids)

    

Now that we've pretty much established how our data should be preprocessed, we want to introduce and train some basic machine learning models 

In [142]:
def train_models(train, labels):
    
    #lassoClf = Lasso()
    #lassoClf.fit(train, labels)
    
    svcClf = SVC(random_state = 260)
    svcClf.fit(train, labels)
    
    rfClf = RandomForestClassifier(random_state = 260)
    rfClf.fit(train, labels)
    
    return [svcClf, rfClf]


  #  return [lassoClf, svcClf, rfClf]
    
    
    
    
    

In [156]:
def Tuner(combined_features, groundtruth):
    '''
    Input: 
        * all features
        * ground truth labels

    Output:
        * tuning grid of (accuracy, sensitivity, specificity) for specified bounds on C, class_weight

    '''
    C = .5
    C_bound = 8
    grid = np.zeros((C_bound, 3))
    for i in range(C_bound):
        grid[i] = EvaluateClassifier(combined_features, groundtruth, C)
        C += .5
    return grid

In [159]:
def EvaluateClassifier(X, Y, C):
    '''
    Input: 
        * Dataset X
        * Labels Y
    Output:
        * outputs performance metrics: accuracy, sens, spec

    Evaluates a linear-kernel support classifer with given penalty input C and melanoma class-weight weight
    with 10-fold cross-validation. We must adjust the class weight to account for the class imbalance inherent 
    to the dataset. 
    '''

    svm_man = SVC(C = C, kernel='linear')

    scores = cross_val_score(svm_man, X, Y, cv=10, scoring='accuracy')
    acc = scores.mean()
    print('acc is:')
    print(acc)

    #------ Compute metrics ---------
    y_predict = cross_val_predict(svm_man, X, Y, cv=10)
    conf_mat = confusion_matrix(Y, y_predict)
    tn, fp, fn, tp = conf_mat.ravel()
    sens = float(tp)/(tp+fn)
    spec = float(tn)/(tn+fp)
    print('sens is:')
    print(sens)
    print('spec is:')
    print(spec)
    print(conf_mat)

    return (acc, sens, spec)

In [143]:
def validate_models(models, test, y_true):
    '''
        input: 
            * models, a list of trained models 
            * test, the test set on which models will be evaluated
            * feature_names, the list of feature columns that are being used 
    '''
    print("Preparing to validate models")
    for model in models:
        y_pred = model.predict(test)
        print(confusion_matrix(y_true, y_pred))
        #print(y_pred)
        print("auroc: " +  str(roc_auc_score(y_true, y_pred)))
        
    print("Done validating models")
    
    

In [144]:
data_parent_dir = "./data_fixed_crop_w_mask"
precomputed_path = "features_matrix.csv"

In [145]:
X_train, y_train, X_test, y_test, all_IDs = preprocess(data_parent_dir, precomputed_path, dim_reduc=None) 

Entering preprocessing
preparing to read images
['00001_LEFT_CC', '00001_LEFT_MLO', '00004_LEFT_CC', '00004_LEFT_MLO', '00004_RIGHT_CC', '00004_RIGHT_MLO', '00009_RIGHT_CC', '00009_RIGHT_MLO', '00015_LEFT_MLO', '00016_LEFT_CC']
786
722
1508
the drop list has: 91
(1508, 23)
(1508, 23)
             label
count  1508.000000
mean      0.478780
std       0.499715
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       1.000000
         convexity     solidity       extent  major_axis_length  \
count  1508.000000  1508.000000  1508.000000        1508.000000   
mean      0.823225     0.863078     0.644197          90.195399   
std       0.151674     0.117002     0.107757          62.893666   
min       0.461828     0.047736     0.016178          31.827394   
25%       0.754172     0.853156     0.620854          55.699932   
50%       0.821002     0.885096     0.662134          68.380669   
75%       0.876884     0.912385     0.699197          93.923356   
max     

In [137]:
models = train_models(X_train, y_train)
validate_models(models, X_test, y_test)

Preparing to validate models
[[102  51]
 [ 78  71]]
auroc: 0.5715883668903804
[[100  53]
 [ 92  57]]
auroc: 0.5180725534061499
Done validating models


Okay, so now that we've seen our models perform like garbage, we're going to try to add in the semantic feature encoding to see if that will help 

In [162]:
def gather_semantic_features(semantics_path):
    semantic_df = pd.read_csv(semantics_path)
    semantic_df.dropna(inplace=True)
    return semantic_df


In [163]:
def encode_categorical_labels(semantic_df, semantic_feature_names):
    '''
        input:
        
        output:
        
    '''
    for feature in semantic_feature_names:
        le = preprocessing.LabelEncoder()
        le.fit(list(semantic_df[feature].astype(str)))
        semantic_df[feature] = le.transform(semantic_df[feature])
    return semantic_df



In [164]:
def one_hot_encoding(semantic_df, semantic_feature_names):
    '''
        input:
        
        output:
        
    '''
    enc = preprocessing.OneHotEncoder(sparse=False)
    semantic_one_hots = enc.fit_transform(semantic_df[semantic_feature_names])
    _, one_hot_length = semantic_one_hots.shape
    return (semantic_one_hots, one_hot_length)



In [165]:
def generate_semantic_df(semantics_path, semantic_feature_names, total_patientIDs):
    
    semantic_df = gather_semantic_features(semantics_path)
    semantic_df = encode_categorical_labels(semantic_df, semantic_feature_names)
    semantic_one_hots, one_hot_length = one_hot_encoding(semantic_df, semantic_feature_names)
   
    has_semantic = [s1 + "_" + s2 + "_" + s3 for (s1, s2, s3) in zip(list(semantic_df['patient_id']), list(semantic_df['side']), list(semantic_df['view']))]

    semantic_encoded_dict = {img:np.zeros(one_hot_length) for (idx, img) in enumerate(total_patientIDs)}

    for img, patient_id in enumerate(has_semantic): 
        if patient_id in semantic_encoded_dict.keys():
            semantic_encoded_dict[patient_id] = semantic_one_hots[img] 
        
    print(one_hot_length)
    print(len(total_patientIDs))
    
    encoded_feature_names = ["one_hot #" + str(x) for x in range(1, one_hot_length+1)]
    semantic_encoded_df = pd.DataFrame.from_dict(semantic_encoded_dict, orient='index', columns=encoded_feature_names)
    return (semantic_encoded_df, encoded_feature_names)
    

Here, we find that there are semantic descriptions of images that do not appear in the h5'd dataset, and also images that appear in this dataset without corresponding semantic descriptions.  

In [166]:
def preprocess2(data_parent_dir, precomputed_path, semantics_path, semantic_feature_names, dim_reduc=None):
    print('Entering preprocessing')
    
    feature_names = []
    
    # reading things in 
    image_df, computed_df, scan_ids = gather_images(data_parent_dir)
    precomputed_df = drop_excess_rows(scan_ids, pd.read_csv(precomputed_path, index_col=0))

    # an initial merge
    feature_df = pd.merge(computed_df, precomputed_df, on=precomputed_df.index)
    
    # zero-centering, normalization of all data within the feature dataframe 
    features = feature_df.values[:,1:]
    new_fts = mean_center_normalize(np.array(features,dtype=np.float32))

    print(image_df.describe())
    print(computed_df.describe())
    print(precomputed_df.describe())
    
    # dimensionality reduction 
    if dim_reduc is not None:
        new_fts = reduce_dimensionality(new_fts, dim_reduc)
        feature_names += ["pc# "+str(x) for x in range(1, dim_reduc+1)]
        feature_df = pd.DataFrame(new_fts, columns=feature_names, index=precomputed_df.index)


        
    # adding back to df 
    else:
        feature_names += list(feature_df)[1:]
        feature_df[feature_names] = new_fts
    
    # get the semantic df
    semantic_df, encoded_feature_names = generate_semantic_df(semantics_path, semantic_feature_names, scan_ids)
    
    feature_names += encoded_feature_names
    
    # joining dfs
    df = pd.merge(image_df, feature_df[feature_names], on=feature_df.index)
    df = pd.merge(df, semantic_df, on=df.index)
    df.info()
    
    train, test = train_test_split(df, test_size=0.2, random_state=260)
    
    return (train[feature_names], train['label'], test[feature_names], test['label'], scan_ids)

In [167]:
semantic_label_name = ['pathology']

In [ ]:
image_df, computed_df, scan_ids = gather_images(data_parent_dir)
semantic_feature_names = ['breast_density', 'abn_num', 'mass_shape', 'mass_margins', 'assessment']
semantics_path = "mass_case_description_train_set.csv"


semantic_df, encoded_feature_names = generate_semantic_df(semantics_path, semantic_feature_names, scan_ids)


preparing to read images


In [161]:
print(semantic_df.index)

Index(['P_00001_LEFT_CC', 'P_00001_LEFT_MLO', 'P_00004_LEFT_CC',
       'P_00004_LEFT_MLO', 'P_00004_RIGHT_CC', 'P_00004_RIGHT_MLO',
       'P_00009_RIGHT_CC', 'P_00009_RIGHT_MLO', 'P_00015_LEFT_MLO',
       'P_00016_LEFT_CC',
       ...
       'P_01550_RIGHT_CC', 'P_01550_RIGHT_MLO', 'P_01551_LEFT_CC',
       'P_01551_LEFT_MLO', 'P_01553_RIGHT_CC', 'P_01553_RIGHT_MLO',
       'P_01555_LEFT_MLO', 'P_01556_LEFT_CC', 'P_01557_RIGHT_CC',
       'P_01557_RIGHT_MLO'],
      dtype='object', length=1508)


In [155]:
data_parent_dir = "./data_fixed_crop_w_mask"
precomputed_path = "features_matrix.csv"
semantics_path = "mass_case_description_train_set.csv"

semantic_feature_names = ['breast_density', 'abn_num', 'mass_shape', 'mass_margins', 'assessment']

X_train, y_train, X_test, y_test = preprocess2(data_parent_dir, 
                                                       precomputed_path, 
                                                       semantics_path, 
                                                       semantic_feature_names, 
                                                       dim_reduc=3)


Entering preprocessing
preparing to read images
['00001_LEFT_CC', '00001_LEFT_MLO', '00004_LEFT_CC', '00004_LEFT_MLO', '00004_RIGHT_CC', '00004_RIGHT_MLO', '00009_RIGHT_CC', '00009_RIGHT_MLO', '00015_LEFT_MLO', '00016_LEFT_CC']
786
722
1508
the drop list has: 91
(1508, 23)
(1508, 23)
             label
count  1508.000000
mean      0.478780
std       0.499715
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       1.000000
         convexity     solidity       extent  major_axis_length  \
count  1508.000000  1508.000000  1508.000000        1508.000000   
mean      0.823225     0.863078     0.644197          90.195399   
std       0.151674     0.117002     0.107757          62.893666   
min       0.461828     0.047736     0.016178          31.827394   
25%       0.754172     0.853156     0.620854          55.699932   
50%       0.821002     0.885096     0.662134          68.380669   
75%       0.876884     0.912385     0.699197          93.923356   
max     

KeyError: "['one_hot #1' 'one_hot #2' 'one_hot #3' 'one_hot #4' 'one_hot #5'\n 'one_hot #6' 'one_hot #7' 'one_hot #8' 'one_hot #9' 'one_hot #10'\n 'one_hot #11' 'one_hot #12' 'one_hot #13' 'one_hot #14' 'one_hot #15'\n 'one_hot #16' 'one_hot #17' 'one_hot #18' 'one_hot #19' 'one_hot #20'\n 'one_hot #21' 'one_hot #22' 'one_hot #23' 'one_hot #24' 'one_hot #25'\n 'one_hot #26' 'one_hot #27' 'one_hot #28' 'one_hot #29' 'one_hot #30'\n 'one_hot #31' 'one_hot #32' 'one_hot #33' 'one_hot #34' 'one_hot #35'\n 'one_hot #36' 'one_hot #37' 'one_hot #38' 'one_hot #39' 'one_hot #40'\n 'one_hot #41' 'one_hot #42' 'one_hot #43' 'one_hot #44' 'one_hot #45'\n 'one_hot #46' 'one_hot #47' 'one_hot #48' 'one_hot #49'] not in index"

In [89]:
models = train_models(X_train, y_train)
validate_models(models, X_test, y_test)

Entering preprocessing
preparing to read images
786
722
1508
preparing to reduce dimensionality
>>> variance explained by each principal component
[4.4473657e-01 1.6824159e-01 9.1169290e-02 5.7367690e-02 5.6748547e-02
 5.2320685e-02 3.6198188e-02 3.1822890e-02 2.2656877e-02 1.7601196e-02
 9.3034180e-03 7.1804966e-03 2.3674807e-03 1.1376145e-03 8.7159796e-04
 2.6753976e-04 6.5508975e-06 1.8152512e-06]
>>> the first principal component
[-0.33582258  0.3129383  -0.04206447 -0.01204051  0.31197217  0.3208049
  0.00518964 -0.33550972 -0.30273733  0.21556541  0.16609216  0.02282244
  0.00318173  0.30442756  0.21560022 -0.05786494  0.3394732   0.22484674]
['1' '2' '3' '4']
['1' '2' '3' '4' '5' '6']
['ARCHITECTURAL_DISTORTION' 'ASYMMETRIC_BREAST_TISSUE'
 'FOCAL_ASYMMETRIC_DENSITY' 'IRREGULAR'
 'IRREGULAR-ARCHITECTURAL_DISTORTION' 'IRREGULAR-FOCAL_ASYMMETRIC_DENSITY'
 'LOBULATED' 'LOBULATED-ARCHITECTURAL_DISTORTION' 'LOBULATED-IRREGULAR'
 'LOBULATED-LYMPH_NODE' 'LOBULATED-OVAL' 'LYMPH_NODE' 'OV

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# Future Work

As mentioned in our introduction, vanilla ML is significantly more interpretable and parsable in terms of the importance of specific variables. In addition, vanilla ML allows us to incoporate expert opinion
and analysis into our models. Because of this, our models do not need to learn specific relationships between masses and malignant character that is already common domain knowledge; most features focus on the 
irregularity of the mass's borders and deviance from sphericality. 


For future work, more feature analysis could potentially result in useful findings and recommendations for pathologists who are examining mammograms. In addition, it would be worthwhile to compare the
performance on this model to a CNN to see which images can be misclassified due to dependence on conventional metrics. Finding examples of such images that are successfully classified under CNN but not 
not correctly classified under vanilla ML approaches would provide excellent examples to provide to pathlogists to warn them about the unconventional nature of different types of masses. In addition, due to both 
lack of time and the limited scope of the data provided, we were unable to to train models based on different orientations of mages. Training different models for different orientions could provide useful
feedback on what variables are more indicative of a mass's malignant character in different kinds of images. 

# References

Bradski, G. (2000). The OpenCV Library. Dr. Dobb's Journal of Software Tools

Britton P, Warwick J, Wallis MG, et al. Measuring the accuracy of diagnostic imaging in symptomatic breast patients: team and individual performance. The British Journal of Radiology. 2012;85(1012):415-422. doi:10.1259/bjr/32906819.

Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

Stéfan van der Walt, Johannes L. Schönberger, Juan Nunez-Iglesias, François Boulogne, Joshua D. Warner, Neil Yager, Emmanuelle Gouillart, Tony Yu and the scikit-image contributors. scikit-image: Image processing in Python. PeerJ 2:e453 (2014) http://dx.doi.org/10.7717/peerj.453

